# Minimal Blockchain in Python

## Defining MinimalChain Class

In [6]:
# We have to import these libraries
import copy
import datetime
import hashlib

## Defining classes

In [56]:
class MinimalChain:
    """
    We create our constructor
    """
    def __init__(self):
        self.blocks = [self.get_genesis_block()]
    
    
    def __eq__(self, other):
        """
        If it is an instance, returns a dictionary
        This is an comparison method
        """
        if isinstance(other, self.__class__):
            return self.__dict__ == other.__dict__
        else:
            return False
    
    
    def get_genesis_block(self):
        """
        We get the genesis block with the following data
        """
        return MinimalBlock(0, 
                            datetime.datetime.utcnow(), 
                            "Genesis",
                            "arbitrary")
    
    
    def add_block(self, data):
        """
        We add another block with the parameter data
        """
        self.blocks.append(MinimalBlock(len(self.blocks),
                                       datetime.datetime.utcnow(),
                                       data, 
                                       self.blocks[len(self.blocks) - 1].hash))
    
    
    def get_chain_size(self):
        """
        We return the size of the chain
        """
        return len(self.blocks) - 1
    
    
    def verify(self, verbose=True):
        """
        We verify that the blockchain is correct and has not been modified
        """
        flag = True
        for i in range(1,len(self.blocks)):
            if not self.blocks[i].verify(): # assume Genesis block integrity
                flag = False
                if verbose:
                    print(f'Wrong data type(s) at block {i}.')
            if self.blocks[i].index != i:
                flag = False
                if verbose:
                    print(f'Wrong block index at block {i}.')
            if self.blocks[i-1].hash != self.blocks[i].previous_hash:
                flag = False
                if verbose:
                    print(f'Wrong previous hash at block {i}.')
            if self.blocks[i].hash != self.blocks[i].hashing():
                flag = False
                if verbose:
                    print(f'Wrong hash at block {i}.')
            if self.blocks[i-1].timestamp >= self.blocks[i].timestamp:
                flag = False
                if verbose:
                    print(f'Backdating at block {i}.')
        return flag
    
    
    def fork(self, head="latest"):
        if head in ["latest", "whole", "all"]:
            """
            We create a deep copy
            """
            return copy.deepcopy(self)
        else:
            c = copy.deepcopy(self)
            c.blocks = c.blocks[0:head + 1]
            return c
    
    
    def get_root(self, chain_2):
        """
        We go through the chains
        """
        min_chain_size = min(self.get_chain_size(), chain_2.get_chain_size())
        for i in range(1, min_chain_size + 1):
            if self.blocks[i] != chain_2.blocks[i]:
                return self.fork(i - 1)
            return self.fork(min_chain_size)
       
        

## Defining the MinimalBlock class

In [57]:
class MinimalBlock:
    """
    We create the constructor with parameters: index, timestamp, data, previous_hash
    """
    def __init__(self, index, timestamp, data, previous_hash):
        self.index = index
        self.timestamp = timestamp
        self.data = data
        self.previous_hash = previous_hash
        self.hash = self.hashing() # hashing comes from the library hashlib
    
    
    def __eq__(self, other):
        """
        If it is an instance, returns a dictionary but, returns False
        This is a comparison method
        """
        if isinstance(other, self.__class__):
            return self.__dict__ == other.__dict__
        else:
            return False
    
    
    def hashing(self):
        """
        We use hashing to encrypt the block
        """
        key = hashlib.sha256()
        key.update(str(self.index).encode("utf-8"))
        key.update(str(self.timestamp).encode("utf-8"))
        key.update(str(self.data).encode("utf-8"))
        key.update(str(self.previous_hash).encode("utf-8"))
        return key.hexdigest()

    
    def verify(self):
        """
        Verify all information in a block
        """
        instances = [self.index, self.timestamp, self.previous_hash, self.hash]
        types = [int, datetime.datetime, str, str]
        if sum(map(lambda inst_, type_: isinstance(inst_, type_), instances, types)) == len(instances):
            return True
        else:
            return False
        

## Testing

In [58]:
c = MinimalChain()
for i in range(1, 20 + 1):
    c.add_block(f"This is block {i} of my first chain")

In [59]:
print(c.blocks[3].timestamp)
print(c.blocks[7].data)
print(c.blocks[9].hash)

2021-02-11 17:50:18.679284
This is block 7 of my first chain
c3af0228fb8815aa449e101ad88285074872508ff084c9813a0514d30821d28e


In [73]:
c_2 = MinimalChain()
for x in range(1, 11 + 1):
    c_2.add_block(f"This is block {i} of my second chain")

print(c_2.blocks[1].timestamp)
print(c_2.blocks[2].data)
print(c_2.blocks[1].hash)


2021-02-11 18:00:22.130330
This is block 20 of my second chain
9f77543ba1d7fa80b50ba239868362a595df7c1504ee6e9e2585884d755e7f44


In [60]:
print(c.get_chain_size())
print(c.verify())

20
True


In [61]:
c_forked = c.fork("latest")
print(c == c_forked)

True


In [62]:
c_forked.add_block("New block for forked chain")
print(c.get_chain_size(), c_forked.get_chain_size())

20 21


## Conflict Testing

In [63]:
c_forked = c.fork("latest")
c_forked.blocks[9].index = -9
c_forked.verify()

Wrong block index at block 9.
Wrong hash at block 9.


False

In [64]:
c_forked = c.fork("latest")
c_forked.blocks[16].timestamp = datetime.datetime(2000, 1, 1, 0, 0, 0, 0)
c_forked.verify()

Wrong hash at block 16.
Backdating at block 16.


False

In [65]:
c_forked = c.fork("latest")
c_forked.blocks[5].previous_hash = c_forked.blocks[1].hash
c_forked.verify()

Wrong previous hash at block 5.
Wrong hash at block 5.


False